### 生成一个示例文档

In [1]:
from langchain_core.documents import Document

document = [
    Document(
        page_content="hahaha",
        metadata={"source": "文档"}
    ),
    Document(
        page_content="hahaha",
        metadata={"source2": "文档2"}
    )
]


### 加载文档

In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))


107


In [4]:
print(docs[0].page_content[0:200])
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F
{'source': 'nke-10k-2023.pdf', 'page': 0}


### splitting

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
len(all_splits)

# for i in range(len(all_splits)):
#     print(all_splits[i].page_content)

516

## embedding

In [11]:
import os
from langchain_openai import OpenAIEmbeddings

os.environ["OPENAI_API_KEY"] = "sk-KlAzhR7lNKTgox4WHHd974EGzaQ0CNJpRMopneI1258s9KNL"
os.environ["OPENAI_BASE_URL"] = "https://api.bianxie.ai/v1"

embeddings_model = OpenAIEmbeddings()


### 虽然文本document的字数不一样，但是生成但向量的维度是一样的

In [19]:
vector1 = embeddings_model.embed_query(all_splits[0].page_content)
vector2 = embeddings_model.embed_query(all_splits[1].page_content)

assert len(vector1) == len(vector2)
print(f"generatied vectors of length: {len(vector1)}")
print(vector1[:10])
print(vector1)




generatied vectors of length: 1536
[-0.00865495577454567, -0.03345286101102829, -0.00998399406671524, -0.005063311196863651, 0.009030976332724094, 0.009439412504434586, -0.028214506804943085, -0.016467107459902763, 0.0030033020302653313, -0.012771732173860073]
[-0.00865495577454567, -0.03345286101102829, -0.00998399406671524, -0.005063311196863651, 0.009030976332724094, 0.009439412504434586, -0.028214506804943085, -0.016467107459902763, 0.0030033020302653313, -0.012771732173860073, 0.01782855950295925, -0.007676005829125643, -0.004427966196089983, 0.01546870730817318, 0.009076357819139957, -0.008207621052861214, -0.0023906477726995945, -0.026489999145269394, 0.0259065181016922, -0.017167283222079277, -0.022807590663433075, 0.007468545809388161, -0.013951659202575684, -0.008700337260961533, -0.008253002539277077, -0.025776857510209084, 0.01404242217540741, -0.023546665906906128, 0.007364816032350063, -0.02529710717499256, -0.009958061389625072, -0.005990396253764629, -0.0221852120012044

In [25]:
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore

index = faiss.IndexFlatL2(len(vector1))


vector_store = FAISS(embedding_function=embeddings_model)



ModuleNotFoundError: No module named 'faiss'